In [1]:
import pandas as pd
import time
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import warnings, sys, os, gc

import surprise as sp
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV, RandomizedSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, SVD, NMF


## Surprise

In [4]:
df_train = pd.read_csv('./data/opiniones_train.csv')
df_test = pd.read_csv('./data/opiniones_test.csv')

In [5]:
trainset, testset = train_test_split(df_train, test_size=0.30)
#trainset = df_train
#testset = df_test
print(trainset.shape, testset.shape)

(29624, 3) (12696, 3)


In [6]:
testset = testset.rename(columns = {'libro': 'id'}, inplace = False)
trainset = trainset.rename(columns = {'libro': 'id'}, inplace = False)

In [7]:
reader = sp.reader.Reader(rating_scale=(1, 10))
data_train= sp.dataset.Dataset.load_from_df(trainset[['id', 'usuario', 'puntuacion']], reader)

### Busqueda hiper parametros

In [8]:
param_grid =  {'min_k': [2, 3, 4, 5, 6, 7, 8], 'k': [10, 15, 20, 25, 30, 35, 40, 45, 50],
                'sim_options': {'name': ["msd", "cosine", "pearson_baseline", "pearson"],
                'min_support': [1, 5],
                'user_based': [False, True]}}

In [29]:
param_grid =  {'min_k': [2, 3, 4, 5, 6, 7, 8], 'k': [10, 15, 20, 25, 30, 35, 40, 45, 50],
                'sim_options': {'name': ["msd", "cosine", "pearson_baseline", "pearson"],
                'user_based': [False]
                }}

In [30]:
t_0=time.time()
knnbasic_rs = RandomizedSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'],n_iter=20, n_jobs=3)
knnbasic_rs.fit(data_train)
#knnbasic_gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=3)
#knnbasic_gs.fit(data_train)
print("KNNBasic")
print('RMSE =', knnbasic_gs.best_params['rmse']) #, 'MAE =', knnbasic_rs.best_params['mae'])
print ("tiempo: ", time.time() - t_0)

exception calling callback for <Future at 0x7fd9e3bce310 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/ignacio/anaconda3/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 551, in apply_async

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
t_0=time.time()
knnmeans_rs = RandomizedSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], n_iter=20, n_jobs=3)
knnmeans_rs.fit(data_train)
#knnbasic_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=3)
#knnbasic_gs.fit(data_train)

print("KNNWithMeans")
print('RMSE =', knnmeans_rs.best_params['rmse']) #, 'MAE =', knnbasic_rs.best_params['mae'])
print ("tiempo: ", time.time() - t_0)

In [38]:
#t_0=time.time()
#knnz_gs     = RandomizedSearchCV(KNNWithZScore, param_grid, measures=['rmse', 'mae'], n_iter=20, #n_jobs=3)
#knnz_gs.fit(data_train)
#print("KNNWithZScore")
#print('RMSE =', knnz_gs.best_params['rmse']) #, 'MAE =', knnbasic_gs.best_params['mae'])
#print ("tiempo: ", time.time() - t_0)

In [22]:
so_Basic = {'name': 'pearson', 'min_support': 1, 'user_based': False}
so_Means = {'name': 'pearson', 'min_support': 1, 'user_based': False}
#so_ZScore = {'name': 'cosine', 'min_support': 1, 'user_based': False}

In [23]:
KNNBasic_algo = sp.prediction_algorithms.knns.KNNBasic(k=15, min_k=8, sim_options=so_Basic)
KNNWithMeans_algo = sp.prediction_algorithms.knns.KNNWithMeans(k=45, min_k=6, sim_options=so_Means)
#KNNWithZScore_algo = sp.prediction_algorithms.knns.KNNWithZScore(k=20, min_k=8, sim_options=so_ZScore)

In [24]:
trainset = data_train.build_full_trainset()

In [25]:
t_0 = time.time()
pred_KNNBasic = KNNBasic_algo.fit(trainset)
print ("tiempo: ", time.time() - t_0)

Computing the pearson similarity matrix...
Done computing similarity matrix.
tiempo:  0.6142673492431641


In [26]:
t_0 = time.time()
pred_KNNWithMeans = KNNWithMeans_algo.fit(trainset)
print ("tiempo: ", time.time() - t_0)

Computing the pearson similarity matrix...
Done computing similarity matrix.
tiempo:  0.6001253128051758


In [8]:
def full_prediction(algo_name,algoritmo,test):
    list_pred = []
    for row in tqdm_notebook(test.itertuples()):
        row_pred = dict()
        uid = row.id
        iid = row.usuario
        #print(uid, iid)
        pred = algoritmo.predict(uid, iid, verbose=False)
        row_pred["id"] = uid
        row_pred["puntuacion"] = np.around(pred.est, 2)
        list_pred.append(row_pred)
    #print(row.ID.astype(int))
    df = pd.DataFrame(list_pred)    
    print(algo_name + " MSE:      %.4f" % np.sqrt(mean_squared_error(test.puntuacion, df.puntuacion)))
    return list_pred

In [28]:
t_0 = time.time()
df_list_pred_KNNBasic = pd.DataFrame(full_prediction("KNNBasic",KNNBasic_algo,testset))
df_list_pred_KNNWithMeans = pd.DataFrame(full_prediction("KNNWithMeans",KNNWithMeans_algo,testset))
print ("tiempo: ", time.time() - t_0)


KNNBasic MSE:      1.7144



KNNWithMeans MSE:      1.6713
tiempo:  1.2819995880126953


In [75]:
df_list_pred_KNN.to_csv("./submission/ignacio_submit_SVD.csv", index=False)

## SVD

In [24]:
# HiperParametros
param_grid_svd = {'n_factors': [50,60,70,80,90,100,110,120], 'n_epochs': [5,10,15,20,25,30], 
              'lr_all': [0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008],
              'reg_all': [0.01,0.02,0.03],
              'biased':[True, False]}

param_grid_nmf = {'n_factors': [5, 10, 15, 20, 25], 'n_epochs': [40, 50, 60],
              'biased':[True, False]}

SVD_algo = SVD()
NMF_algo = NMF()

In [25]:
t_0 = time.time()
svd_rs = RandomizedSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'],n_iter=20, n_jobs=2)
svd_rs.fit(data_train)
print("SVD")
print('RMSE =', svd_rs.best_params['rmse']) 
print ("tiempo: ", time.time() - t_0)

SVD
RMSE = {'n_factors': 120, 'n_epochs': 15, 'lr_all': 0.004, 'reg_all': 0.01, 'biased': True}
tiempo:  88.99728417396545


In [26]:
t_0 = time.time()
nmf_rs = RandomizedSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'],n_iter=15, n_jobs=2)
nmf_rs.fit(data_train)
print("NMF")
print('RMSE =', nmf_rs.best_params['rmse']) 
print ("tiempo: ", time.time() - t_0)

NMF
RMSE = {'n_factors': 5, 'n_epochs': 40, 'biased': True}
tiempo:  93.26794981956482


In [20]:
SVD_algo = SVD(n_factors= 120, n_epochs= 15, lr_all= 0.004, reg_all= 0.01, biased=True)
NMF_algo = NMF(n_factors= 25, n_epochs= 50, biased=True)

In [21]:
trainset_model = data_train.build_full_trainset()
t_0 = time.time()
pred_SVD = SVD_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)
t_0 = time.time()
pred_NMF = NMF_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)

tiempo:  0.8231487274169922
tiempo:  3.6792430877685547


In [22]:
print(len(testset))

12696


In [23]:
df_list_pred_SVD = pd.DataFrame(full_prediction("SVD",SVD_algo,testset))
df_list_pred_NMF = pd.DataFrame(full_prediction("NMF",NMF_algo,testset))


SVD MSE:      1.5847



NMF MSE:      1.8399
